<a href="https://colab.research.google.com/github/nidhib99/NLP/blob/master/Classwork/NLP_question%26answers_J006.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Colab library to upload files to notebook
from google.colab import files

# Install Kaggle library
!pip install -q kaggle

In [2]:
# Upload kaggle API key file
uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [0]:
!cp kaggle.json ~/.kaggle/kaggle.json
#!mkdir ~/.kaggle
#!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json

In [6]:
!kaggle datasets download -d stanfordu/stanford-question-answering-dataset

100% 8.73M/8.73M [00:00<00:00, 31.2MB/s]



In [7]:
from zipfile import ZipFile 
  
file_name = "/content/stanford-question-answering-dataset.zip"
  
with ZipFile(file_name, 'r') as zip: 
    zip.printdir() 
    zip.extractall() 

File Name                                             Modified             Size
dev-v1.1.json                                  2019-11-17 22:02:26      4854279
train-v1.1.json                                2019-11-17 22:02:26     30288272


In [0]:
import pandas as pd
import json
import numpy as np


In [0]:
def squad_json_to_dataframe_train(input_file_path, record_path = ['data','paragraphs','qas','answers'],
                           verbose = 1):
    if verbose:
        print("Reading the json file")    
    file = json.loads(open(input_file_path).read())
    if verbose:
        print("processing...")
    # parsing different level's in the json file
    js = pd.io.json.json_normalize(file , record_path )
    m = pd.io.json.json_normalize(file, record_path[:-1] )
    r = pd.io.json.json_normalize(file,record_path[:-2])
    
    #combining it into single dataframe
    idx = np.repeat(r['context'].values, r.qas.str.len())
    ndx  = np.repeat(m['id'].values,m['answers'].str.len())
    m['context'] = idx
    js['q_idx'] = ndx
    main = pd.concat([ m[['id','question','context']].set_index('id'),js.set_index('q_idx')],1,sort=False).reset_index()
    main['c_id'] = main['context'].factorize()[0]
    if verbose:
        print("shape of the dataframe is {}".format(main.shape))
        print("Done")
    return main

In [10]:
input_file_path = '/content/train-v1.1.json'
record_path = ['data','paragraphs','qas','answers']
train = squad_json_to_dataframe_train(input_file_path=input_file_path,record_path=record_path)

Reading the json file
processing...
shape of the dataframe is (87599, 6)
Done


In [11]:
train

,index,question,context,answer_start,text,c_id
0,5733be284776f41900661182,To whom did the Virgin Mary allegedly appear i...,"Architecturally, the school has a Catholic cha...",515,Saint Bernadette Soubirous,0
1,5733be284776f4190066117f,What is in front of the Notre Dame Main Building?,"Architecturally, the school has a Catholic cha...",188,a copper statue of Christ,0
2,5733be284776f41900661180,The Basilica of the Sacred heart at Notre Dame...,"Architecturally, the school has a Catholic cha...",279,the Main Building,0
3,5733be284776f41900661181,What is the Grotto at Notre Dame?,"Architecturally, the school has a Catholic cha...",381,a Marian place of prayer and reflection,0
4,5733be284776f4190066117e,What sits on top of the Main Building at Notre...,"Architecturally, the school has a Catholic cha...",92,a golden statue of the Virgin Mary,0
...,...,...,...,...,...,...
87594,5735d259012e2f140011a09d,In what US state did Kathmandu first establish...,"Kathmandu Metropolitan City (KMC), in order to...",229,Oregon,18890
87595,5735d259012e2f140011a09e,What was Yangon previously known as?,"Kathmandu Metropolitan City (KMC), in order to...",414,Rangoon,18890
87596,5735d259012e2f140011a09f,With what Belorussian city does Kathmandu have...,"Kathmandu Metropolitan City (KMC), in order to...",476,Minsk,18890
87597,5735d259012e2f140011a0a0,In what year did Kathmandu create its initial ...,"Kathmandu Metropolitan City (KMC), in order to...",199,1975,18890


In [12]:
import re
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from gensim.models import Word2Vec
from string import digits,punctuation 
from nltk.tokenize import word_tokenize

nltk.download('punkt')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords') 
nltk.download('averaged_perceptron_tagger')

remdigits = str.maketrans('', '', digits)
stopword = set(stopwords.words('english'))

def preprocess(str):
  str = str.lower()
  str = str.translate(remdigits)
  str = re.sub(r'[^\w\s]',' ',str)
  return str

def tokens(str):
  token = word_tokenize(str)
  words = [word for word in token if word not in stopword and word not in punctuation]
  return words

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [0]:
df2 = train['question']
df2 = pd.DataFrame(df2)
df2['B'] = df2['question'].apply(preprocess)
df2['B'] = df2['question'].apply(tokens)
final = df2['question'].apply(tokens).tolist()

In [14]:
final

[['To', 'Virgin', 'Mary', 'allegedly', 'appear', '1858', 'Lourdes', 'France'],
 ['What', 'front', 'Notre', 'Dame', 'Main', 'Building'],
 ['The',
  'Basilica',
  'Sacred',
  'heart',
  'Notre',
  'Dame',
  'beside',
  'structure'],
 ['What', 'Grotto', 'Notre', 'Dame'],
 ['What', 'sits', 'top', 'Main', 'Building', 'Notre', 'Dame'],
 ['When', 'Scholastic', 'Magazine', 'Notre', 'dame', 'begin', 'publishing'],
 ['How', 'often', 'Notre', 'Dame', "'s", 'Juggler', 'published'],
 ['What', 'daily', 'student', 'paper', 'Notre', 'Dame', 'called'],
 ['How', 'many', 'student', 'news', 'papers', 'found', 'Notre', 'Dame'],
 ['In',
  'year',
  'student',
  'paper',
  'Common',
  'Sense',
  'begin',
  'publication',
  'Notre',
  'Dame'],
 ['Where', 'headquarters', 'Congregation', 'Holy', 'Cross'],
 ['What', 'primary', 'seminary', 'Congregation', 'Holy', 'Cross'],
 ['What', 'oldest', 'structure', 'Notre', 'Dame'],
 ['What', 'individuals', 'live', 'Fatima', 'House', 'Notre', 'Dame'],
 ['Which', 'prize', '

In [0]:
df2 = df2.drop('question', 1)

In [27]:
df2

,B
0,"[To, Virgin, Mary, allegedly, appear, 1858, Lo..."
1,"[What, front, Notre, Dame, Main, Building]"
2,"[The, Basilica, Sacred, heart, Notre, Dame, be..."
3,"[What, Grotto, Notre, Dame]"
4,"[What, sits, top, Main, Building, Notre, Dame]"
...,...
87594,"[In, US, state, Kathmandu, first, establish, i..."
87595,"[What, Yangon, previously, known]"
87596,"[With, Belorussian, city, Kathmandu, relations..."
87597,"[In, year, Kathmandu, create, initial, interna..."


In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
def document(doc):
    return doc
vectorizer = TfidfVectorizer(analyzer='word',tokenizer=document,preprocessor=document,token_pattern=None)
vectors = vectorizer.fit_transform(df2['B'])
feature_names = vectorizer.get_feature_names()
#dense = vectors.todense()
#denselist = dense.tolist()
#df3 = pd.DataFrame(denselist, columns=feature_names)

In [29]:
feature_names

["''",
 "''Beautiful",
 "''Carmen",
 "''Deja",
 "''The",
 "''narrow",
 "'00s",
 "'04",
 "'06",
 "'08",
 "'1941",
 "'50s",
 "'60s",
 "'70s",
 "'73",
 "'78s",
 "'80s",
 "'90s",
 "'97",
 "'98",
 "'A",
 "'Abbas",
 "'Abdu'l-Hamid",
 "'Abdullah",
 "'Abubacer",
 "'Against",
 "'Alciphron",
 "'Ali",
 "'Always",
 "'Amblin",
 "'American",
 "'An",
 "'Armageddon",
 "'Armenia",
 "'Armenians",
 "'Arrival",
 "'Articulata",
 "'Awakenings",
 "'Batman",
 "'Batteries",
 "'Best",
 "'Bible",
 "'Bonfire",
 "'Bridge",
 "'Britain",
 "'Bronx",
 "'Buena",
 "'Catch",
 "'Chaoxian",
 "'China",
 "'Christian",
 "'City",
 "'Continental",
 "'Discoveryland",
 "'Duel",
 "'Early",
 "'Ebn",
 "'Em",
 "'Empire",
 "'Escape",
 "'Falling",
 "'Fort",
 "'From",
 "'Fuga",
 "'German",
 "'Giant",
 "'God",
 "'Great",
 "'Ideals",
 "'Indiana",
 "'Interstellar",
 "'Into",
 "'Jaws",
 "'Keh",
 "'King",
 "'LA",
 "'Lectures",
 "'Lego",
 "'Letter",
 "'Lincoln",
 "'Little",
 "'Man",
 "'Melting",
 "'Memoirs",
 "'Men",
 "'Minority",
 "'Modern",

In [19]:
len(feature_names)

44572

In [20]:
!pip install gensim

In [0]:
from gensim import models

In [0]:
features = feature_names[100:110]

In [23]:
from itertools import permutations 
perm = []
perm = list(permutations(features, 2))
print(perm)

[("'Ready", "'Reflections"), ("'Ready", "'Robopocalypse"), ("'Ready", "'Roller"), ("'Ready", "'S"), ("'Ready", "'Savage'/"), ("'Ready", "'Saving"), ("'Ready", "'Schindler"), ("'Ready", "'Scullers"), ("'Ready", "'Seven"), ("'Reflections", "'Ready"), ("'Reflections", "'Robopocalypse"), ("'Reflections", "'Roller"), ("'Reflections", "'S"), ("'Reflections", "'Savage'/"), ("'Reflections", "'Saving"), ("'Reflections", "'Schindler"), ("'Reflections", "'Scullers"), ("'Reflections", "'Seven"), ("'Robopocalypse", "'Ready"), ("'Robopocalypse", "'Reflections"), ("'Robopocalypse", "'Roller"), ("'Robopocalypse", "'S"), ("'Robopocalypse", "'Savage'/"), ("'Robopocalypse", "'Saving"), ("'Robopocalypse", "'Schindler"), ("'Robopocalypse", "'Scullers"), ("'Robopocalypse", "'Seven"), ("'Roller", "'Ready"), ("'Roller", "'Reflections"), ("'Roller", "'Robopocalypse"), ("'Roller", "'S"), ("'Roller", "'Savage'/"), ("'Roller", "'Saving"), ("'Roller", "'Schindler"), ("'Roller", "'Scullers"), ("'Roller", "'Seven"),

In [0]:
model = models.Word2Vec(perm)

In [25]:
vocabulary = model.wv.vocab
print(vocabulary)

{"'Ready": <gensim.models.keyedvectors.Vocab object at 0x7f96babc37b8>, "'Reflections": <gensim.models.keyedvectors.Vocab object at 0x7f96babcccc0>, "'Robopocalypse": <gensim.models.keyedvectors.Vocab object at 0x7f968736c630>, "'Roller": <gensim.models.keyedvectors.Vocab object at 0x7f968736c908>, "'S": <gensim.models.keyedvectors.Vocab object at 0x7f968736c7f0>, "'Savage'/": <gensim.models.keyedvectors.Vocab object at 0x7f968736c9e8>, "'Saving": <gensim.models.keyedvectors.Vocab object at 0x7f968736ca20>, "'Schindler": <gensim.models.keyedvectors.Vocab object at 0x7f968736c400>, "'Scullers": <gensim.models.keyedvectors.Vocab object at 0x7f968736ca58>, "'Seven": <gensim.models.keyedvectors.Vocab object at 0x7f968736c940>}


In [26]:
for i in perm:
  print((i[0],i[1]),model.similarity(i[0],i[1]))

("'Ready", "'Reflections") 0.063318625
("'Ready", "'Robopocalypse") 0.04494766
("'Ready", "'Roller") -0.133522
("'Ready", "'S") -0.07972005
("'Ready", "'Savage'/") -0.16240622
("'Ready", "'Saving") 0.08439882
("'Ready", "'Schindler") 0.06589291
("'Ready", "'Scullers") 0.12879589
("'Ready", "'Seven") -0.10404023
("'Reflections", "'Ready") 0.063318625
("'Reflections", "'Robopocalypse") 0.07649088
("'Reflections", "'Roller") -0.22401191
("'Reflections", "'S") -0.02555074
("'Reflections", "'Savage'/") 0.12265146
("'Reflections", "'Saving") 0.02565137
("'Reflections", "'Schindler") 0.075217836
("'Reflections", "'Scullers") 0.010180908
("'Reflections", "'Seven") 0.0377996
("'Robopocalypse", "'Ready") 0.04494766
("'Robopocalypse", "'Reflections") 0.07649088
("'Robopocalypse", "'Roller") -0.13684095
("'Robopocalypse", "'S") 0.11232662
("'Robopocalypse", "'Savage'/") 0.016543917
("'Robopocalypse", "'Saving") -0.08409996
("'Robopocalypse", "'Schindler") 0.120500356
("'Robopocalypse", "'Scullers"

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
